In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import openpyxl # This is important for reading excel files! 


# Read the Excel file
df = pd.read_excel(r'C:\Users\SHAWN\OneDrive\Documents\Tablaeu Datasets\my_spotify_data\Spot Data\Streaming_History_Audio_2013-2018_0.xlsx')

# Drop null values in `spotify_track_uri`
df.dropna(subset=['spotify_track_uri'], inplace=True)

# Drop duplicates in `spotify_track_uri`
track_uris = df['spotify_track_uri'].unique()

# Initialize Spotipy with your credentials
auth_manager = SpotifyClientCredentials(client_id='50f51cebf52c4c42a21cf0b60e16f9b0',
                                       client_secret='f1ee0c620a68480b9fb3a8b8ce98de5c')
sp = spotipy.Spotify(auth_manager=auth_manager)

# Function to get genre for a track
def get_genre(track_uri):
    # Filter the dataframe to get the track name
    track_name_series = df.loc[df['spotify_track_uri'] == track_uri, 'master_metadata_track_name']

    if not track_name_series.empty:
        track_name = track_name_series.values[0]
        print(f"Processing: {track_name}")
    else:
        print(f"Track not found for URI: {track_uri}")

    try:
        track_info = sp.track(track_uri)
        artist_uri = track_info['artists'][0]['uri']
        artist_info = sp.artist(artist_uri)
        return artist_info['genres'][0] if artist_info['genres'] else 'Unknown'
    except Exception as e:
        print(f"Error fetching genre for {track_uri}: {e}")
        return 'Unknown'

# Get genres for all tracks
genres = []
for track_uri in track_uris:
    genres.append(get_genre(track_uri))
    # Add a small delay to avoid rate limiting
    time.sleep(0.1)

# Create a new dataframe with track URIs and genres
genre_df = pd.DataFrame({'spotify_track_uri': track_uris, 'Genre': genres})

# Merge the genre dataframe with the original dataframe
merged_df = df.merge(genre_df, on='spotify_track_uri', how='left')

# Write the merged dataframe to a new csv file
merged_df.to_csv('streaming_history_with_genres.csv', index=False)

print("Script completed. Data saved to streaming_history_with_genres.csv")

Processing: Thrift Shop (feat. Wanz)
